In [26]:
import cv2
import torch
from transformers import ViTImageProcessor, ViTForImageClassification

# Initialize the image processor and model
processor = ViTImageProcessor.from_pretrained('google/vit-base-patch16-224-in21k')
model = ViTForImageClassification.from_pretrained('ChrisGuarino/cat_ds')  # Replace with your model
model.eval()

# Define your class labels
class_labels = ['Prim', 'Rupe', 'No Cat']  # Replace with your actual labels

# Start the webcam
cap = cv2.VideoCapture(0)

#Confidence Threshold
confidence_threshold = 0.5  # Define a threshold

while True:
    ret, frame = cap.read()

    if ret:
        # Preprocess the frame
        frame_resized = cv2.resize(frame, (224, 224))
        frame_rgb = cv2.cvtColor(frame_resized, cv2.COLOR_BGR2RGB)
        inputs = processor(images=frame_rgb, return_tensors="pt")

        # Get predictions
        with torch.no_grad():
            predictions = model(**inputs).logits

            # Convert predictions to probabilities and get the highest probability class
            probabilities = torch.nn.functional.softmax(predictions, dim=-1)
            print(probabilities)
            confidences, predicted_class_idx = torch.max(probabilities, dim=-1)
            predicted_class = class_labels[predicted_class_idx]#Something with +1 to shift the labels if we add a No Cat label

        # Check if confidence is above the threshold
        if confidences.item() < confidence_threshold:
            label = 'No Cat'
            confidence = 0
        else:
            label = class_labels[predicted_class_idx.item()]  # +1 to account for 'No Cat'
            confidence = confidences.item()

        # Prepare the display text
        display_text = f'{label} ({confidence:.2f})'

        # Display the prediction on the frame
        cv2.putText(frame_resized, display_text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

        # Display the resulting frame
        cv2.imshow('Frame', frame_resized)

        # Break the loop with 'q'
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    else:
        break

# Release the capture when done
cap.release()
cv2.destroyAllWindows()


tensor([[0.8902, 0.1098]])
tensor([[0.8682, 0.1318]])
tensor([[0.8877, 0.1123]])
tensor([[0.9001, 0.0999]])
tensor([[0.8739, 0.1261]])
tensor([[0.8118, 0.1882]])
tensor([[0.9142, 0.0858]])
tensor([[0.9240, 0.0760]])
tensor([[0.9245, 0.0755]])
tensor([[0.9126, 0.0874]])
tensor([[0.8985, 0.1015]])
tensor([[0.9136, 0.0864]])
tensor([[0.9192, 0.0808]])
tensor([[0.9016, 0.0984]])
tensor([[0.8686, 0.1314]])
tensor([[0.8764, 0.1236]])
tensor([[0.8817, 0.1183]])
tensor([[0.8649, 0.1351]])
tensor([[0.8895, 0.1105]])
tensor([[0.8747, 0.1253]])
tensor([[0.8264, 0.1736]])
tensor([[0.8448, 0.1552]])
tensor([[0.7822, 0.2178]])
tensor([[0.8631, 0.1369]])
tensor([[0.8402, 0.1598]])
tensor([[0.8514, 0.1486]])
tensor([[0.8607, 0.1393]])
tensor([[0.9001, 0.0999]])
tensor([[0.8798, 0.1202]])
tensor([[0.8788, 0.1212]])
tensor([[0.8715, 0.1285]])
tensor([[0.8304, 0.1696]])
tensor([[0.8507, 0.1493]])
tensor([[0.9108, 0.0892]])
tensor([[0.0291, 0.9709]])
tensor([[0.0225, 0.9775]])
tensor([[0.0220, 0.9780]])
t